In [11]:
import boto3
import tempfile
import pyspark

In [9]:
s3 = boto3.client('s3', 
        endpoint_url='http://minio:9000',
        aws_access_key_id='admin',
        aws_secret_access_key='password',
        aws_session_token=None,
        config=boto3.session.Config(signature_version='s3v4'),
        verify=False
    )
BUCKET='rawcsv'

In [13]:
%%sql

CREATE DATABASE IF NOT EXISTS nessie

25/04/06 06:57:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [15]:
# download the file
data = s3.get_object(Bucket=BUCKET, Key='df_open_2023.csv')
# print(data['Body'].read().decode('utf-8').splitlines()[0])
# create a temporary file
with tempfile.NamedTemporaryFile(mode='w',suffix='.csv') as tf:
    # write the downloaded data to the temporary file
    tf.write(data['Body'].read().decode('utf-8'))

    df = spark.read.format("csv").option("header", "true").load(tf.name)
    df.write.saveAsTable("nessie.df_open_2023_lesson")


    # close and delete the temporary file when we're done with it
    tf.close()

In [16]:
%%sql

SELECT COUNT(*) as cnt
FROM nessie.df_open_2023_lesson

cnt
302231
